In [5]:
using Statistics, Distributions, Plots, CSV, DataFrames

# Function to calculate LOLP given imbalance data and reserve level R
function calculate_lolp(imbalances, R)
    num_exceeding_R = count(imbalances .> R)
    total_imbalances = length(imbalances)
    return num_exceeding_R / total_imbalances
end

# Function to compute ORDC points for a range of reserve levels
function compute_ordc(imbalances, voll, reserve_levels)
    ordc_points = []
    for R in reserve_levels
        lolp = calculate_lolp(imbalances, R)
        ordc_value = lolp * voll  # ORDC value using formula ORDC(Rt) = LOLP(Rt) * VOLL
        push!(ordc_points, (R, ordc_value))
    end
    return ordc_points
end

df = CSV.read("Imbalance_202201010000-202301010000.csv", DataFrame)
imbalances = df[!, "Total Imbalance [MWh] - SCA|FR"]

# Remove empty strings and specific string values ("N/A")
filtered_imbalances = filter(x -> x != "" && x != "N/A", imbalances)

# Convert filtered strings to numerical values (assuming all valid strings are numeric)
imbalances = parse.(Float64, filtered_imbalances)

voll = 5000.0  # Value of Lost Load (VOLL) in $/MWh
reserve_levels = 0.0:10.0:1100.0  # Range of reserve levels to evaluate (in MW)

# Compute ORDC points
ordc_points = compute_ordc(imbalances, voll, reserve_levels)

# Extract x (reserve levels) and y (ORDC values) from ORDC points
x = [point[1] for point in ordc_points]
y = [point[2] for point in ordc_points]

# Plot ORDC curve
ORDC_curve=plot(x, y, xlabel="Reserve Level (MW)", ylabel="Operating Reserve Demand Cost", label="ORDC", legend=:topleft)
title!("Operating Reserve Demand Curve (ORDC)")
savefig(ORDC_curve, "ORDC.png")


"c:\\Users\\Mary Jo\\Desktop\\UCL\\Master\\Q2\\QEE\\LINMA2415_QEE_Project\\ORDC.png"